In [1]:
pip install pyowm

Note: you may need to restart the kernel to use updated packages.


## Importing Libraries

In [2]:
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
import pandas as pd

In [3]:
OWM_key = 'your key'
owm = OWM(OWM_key)
mgr = owm.weather_manager()

In [4]:
# current weather in Berlin
observation = mgr.weather_at_place('Berlin,Ger')
w = observation.weather

In [5]:
w.status

'Clouds'

In [6]:
w.detailed_status

'broken clouds'

In [7]:
w.wind()

{'speed': 12.96, 'deg': 203, 'gust': 24.14}

In [8]:
w.humidity

72

In [9]:
w.temperature('celsius')

{'temp': 12.79,
 'temp_max': 13.99,
 'temp_min': 11.16,
 'feels_like': 12.0,
 'temp_kf': None}

In [10]:
w.rain

{}

In [11]:
w.heat_index

In [12]:
w.clouds

75

In [13]:
w.pressure

{'press': 973, 'sea_level': None}

In [14]:
# empty sunrise_time() than unix time format, use 'iso' or 'date'
w.sunrise_time(timeformat='iso')

'2022-04-07 04:26:59+00:00'

In [15]:
w.sunset_time(timeformat='iso')

'2022-04-07 17:49:41+00:00'

In [16]:
import requests 
response = requests.get('http://api.openweathermap.org/data/2.5/forecast/?q=Berlin,DE&appid=yourkey8&units=metric&lang=en')
from IPython.display import JSON
response.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1649343600,
   'main': {'temp': 12.47,
    'feels_like': 11.55,
    'temp_min': 11.03,
    'temp_max': 12.47,
    'pressure': 977,
    'sea_level': 977,
    'grnd_level': 977,
    'humidity': 68,
    'temp_kf': 1.44},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 83},
   'wind': {'speed': 8.8, 'deg': 253, 'gust': 17},
   'visibility': 10000,
   'pop': 1,
   'rain': {'3h': 1.91},
   'sys': {'pod': 'd'},
   'dt_txt': '2022-04-07 15:00:00'},
  {'dt': 1649354400,
   'main': {'temp': 10.67,
    'feels_like': 9.46,
    'temp_min': 9.41,
    'temp_max': 10.67,
    'pressure': 980,
    'sea_level': 980,
    'grnd_level': 978,
    'humidity': 64,
    'temp_kf': 1.26},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'clouds': {'all': 81},
   'wind': {'speed': 8.77, 'deg': 246, 'gust': 17.78},
   'visib

In [17]:
# request one-call api - get object that contains all weather info
one_call = mgr.one_call(lat=52.5244, lon=13.4105, exclude='minutely', units='metric')
one_call.forecast_hourly

[<pyowm.weatherapi25.weather.Weather - reference_time=2022-04-07 12:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2022-04-07 13:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2022-04-07 14:00:00+00:00, status=clouds, detailed_status=overcast clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2022-04-07 15:00:00+00:00, status=rain, detailed_status=light rain>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2022-04-07 16:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2022-04-07 17:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2022-04-07 18:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2022-04-07 19:00:00+00:00, status=clouds, detailed_st

In [18]:
one_call.current.temp

{'temp': 12.67, 'feels_like': 11.87}

In [19]:
one_call

<pyowm.weatherapi25.one_call.OneCall - lat=52.5244, lon=13.4105, retrieval_time=1649334033>

## Wheather API

#### Using pyowm library

In [20]:

OWM_key = 'your key'

city = "Berlin"
country = "DE"

owm = OWM(OWM_key)
mgr = owm.weather_manager()

# Will it be clear tomorrow at this time in Milan (Italy) ?
forecast = mgr.forecast_at_place(city + ',' + country, '3h')
# answer = forecast.will_be_clear_at(timestamps.tomorrow())

from numpy import nan

def forecast_extraction(forecast_object): 
    """
    Given a forecast object from Open weathor Pyhton API, 
    return a structured dataframe. 

    :param: forecast object
    :returns: a dataframe with specific weather information
    """
    forecast_df = pd.json_normalize({
        'time' : forecast_object.reference_time(timeformat='date'), 
        'temperature' : forecast_object.temperature(unit='celsius'), 
        'precipitation_probabily' : forecast_object.precipitation_probability, 
        'rain' : forecast_object.rain, 
        'humidity' : forecast_object.humidity, 
        'status' : forecast_object.status, 
        'snow' : [nan if forecast_object.snow is None else forecast_object.snow], 
        'wind' : forecast_object.wind('km_hour')
    })

    return(forecast_df)

weather_forecast = pd.concat(
    [forecast_extraction(weather) for weather in forecast.forecast.weathers]
)

weather_forecast.head()

,time,precipitation_probabily,humidity,status,snow,temperature.temp,temperature.temp_kf,temperature.temp_max,temperature.temp_min,temperature.feels_like,rain.3h,wind.speed,wind.deg,wind.gust
0,2022-04-07 15:00:00+00:00,1.00,70,Rain,[{}],12.22,-271.96,12.22,11.03,11.33,1.91,31.680,253,61.200
0,2022-04-07 18:00:00+00:00,1.00,65,Rain,[{}],10.55,-272.01,10.55,9.41,9.36,0.14,31.572,246,64.008
0,2022-04-07 21:00:00+00:00,0.66,65,Rain,[{}],8.26,-273.15,8.26,8.26,4.02,0.37,34.236,243,65.016
0,2022-04-08 00:00:00+00:00,0.59,69,Rain,[{}],7.09,-273.15,7.09,7.09,2.22,0.70,38.160,250,71.388
0,2022-04-08 03:00:00+00:00,0.93,76,Rain,[{}],5.40,-273.15,5.40,5.40,0.19,0.61,34.992,284,58.572


In [21]:
#Connecting to OWM API

response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
response

<Response [200]>

In [22]:
forecast_api = response.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city + ',' + country
    weather_info.append(weather_hour)
    
weather_data = pd.DataFrame(weather_info)
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-04-07 15:00:00,12.47,8.80,1.00,1.91,0,"Berlin,DE"
1,2022-04-07 18:00:00,10.67,8.77,1.00,0.14,0,"Berlin,DE"
2,2022-04-07 21:00:00,8.26,9.51,0.66,0.37,0,"Berlin,DE"
3,2022-04-08 00:00:00,7.09,10.60,0.59,0.70,0,"Berlin,DE"
4,2022-04-08 03:00:00,5.40,9.72,0.93,0.61,0,"Berlin,DE"


## Update data into database

### `sqlalchemy`

#### Establish the connection

In [23]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install PyMySQL

Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import sqlalchemy
import pymysql

#### Update data in AWS DATABASE Instance

In [26]:
schema="gans"
host="you host name"
user="your username"
password="your password"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

(
 weather_data.assign(datetime = lambda x: pd.to_datetime(x['datetime'])).to_sql('weather', if_exists='append', con=con, index=False)
)